## Data Cleaning

In [1]:
# Import relevant libraries
import pandas as pd
import nfl_data_py as nfl

In [2]:
# Read in the web scraped data
df2 = pd.read_excel('./Data/sportsrefscrape.xlsx')
df2

,Draft_Yr,Rnd,Pick,Tm,Player,Pos,Age,To,College/Univ,Player_NFL_Link,...,pass_cmp,pass_att,pass_cmp_pct,pass_yds,pass_yds_per_att,adj_pass_yds_per_att,pass_td,pass_int,pass_rating,g
0,2013,1,1,KAN,Eric Fisher,T,22.0,2021.0,Central Michigan,/players/F/FishEr00.htm,...,None,None,None,None,None,None,None,None,None,None
1,2013,1,2,JAX,Luke Joeckel,T,21.0,2017.0,Texas A&M,/players/J/JoecLu00.htm,...,None,None,None,None,None,None,None,None,None,None
2,2013,1,3,MIA,Dion Jordan,DE,23.0,2020.0,Oregon,/players/J/JordDi00.htm,...,None,None,None,None,None,None,None,None,None,NaN
3,2013,1,4,PHI,Lane Johnson,T,23.0,2022.0,Oklahoma,/players/J/JohnLa01.htm,...,None,None,None,None,None,None,None,None,None,None
4,2013,1,5,DET,Ezekiel Ansah,DE,24.0,2020.0,BYU,/players/A/AnsaEz00.htm,...,None,None,None,None,None,None,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553,2022,7,258,GNB,Samori Toure,WR,24.0,2022.0,Nebraska,/players/T/TourSa00.htm,...,None,None,None,None,None,None,None,None,None,None
2554,2022,7,259,KAN,Nazeeh Johnson,SAF,24.0,2022.0,Marshall,/players/J/JohnNa01.htm,...,None,None,None,None,None,None,None,None,None,NaN
2555,2022,7,260,LAC,Zander Horvath,RB,23.0,2022.0,Purdue,/players/H/HorvZa00.htm,...,None,None,None,None,None,None,None,None,None,None
2556,2022,7,261,LAR,AJ Arcuri,OT,25.0,2022.0,Michigan St.,/players/A/ArcuAJ00.htm,...,None,None,None,None,None,None,None,None,None,None


In [3]:
# Call NFL Players Python pacakge
player = nfl.import_players()

# Find all players who were drafted in 2013 or later
draft_players = player.loc[(player.entry_year > 2012) & (player.draft_number >= 1)].sort_values(by=['entry_year','draft_number'])

# Select relevant columns of draft players
draft_player = draft_players[['display_name','position_group','height','weight','entry_year','college_conference','draft_number']]

In [4]:
draft_player

,display_name,position_group,height,weight,entry_year,college_conference,draft_number
6475,Eric Fisher,OL,6-7,315,2013.0,Mid-American Conference,1.0
12179,Luke Joeckel,OL,6-6,307,2013.0,Southeastern Conference,2.0
5746,Dion Jordan,DL,6-6,275,2013.0,Pacific Twelve Conference,3.0
11713,Lane Johnson,OL,6-6,317,2013.0,Big Twelve Conference,4.0
6726,Ezekiel Ansah,DL,6-5,275,2013.0,Independent,5.0
...,...,...,...,...,...,...,...
16258,Samori Toure,WR,6-3,190,2022.0,Big Ten Conference,258.0
14001,Nazeeh Johnson,DB,6-2,189,2022.0,Conference USA,259.0
19237,Zander Horvath,RB,6-3,230,2022.0,Big Ten Conference,260.0
1,A.J. Arcuri,OL,6-7,320,2022.0,Big Ten Conference,261.0


### Find all columns that need to become numeric

In [5]:
# Remove all columns that should stay as objects from df2

cols = []
cols_remove = ['Tm','Player','Pos','College/Univ','Player_NFL_Link','Player_NCAA_Link','display_name','position_group','college_conference']
num_cols = []
for col in df2:
    cols.append(col)
for i in cols:
    if i not in cols_remove:
        num_cols.append(i)

In [6]:
# Apply pd.to_numeric for all of the columns need to be converted
df2[num_cols] = df2[num_cols].apply(pd.to_numeric,errors='coerce', axis=1)

### Merge Scraped Data with NFL Player Python Package

In [7]:
df = df2.merge(draft_player,how='left',left_on=['Draft_Yr','Pick','Player'],right_on=['entry_year','draft_number','display_name'])

In [8]:
# Check for mismerges by finding duplicated players
df[df.duplicated(subset=['Player','Draft_Yr'])]

,Draft_Yr,Rnd,Pick,Tm,Player,Pos,Age,To,College/Univ,Player_NFL_Link,...,pass_int,pass_rating,g,display_name,position_group,height,weight,entry_year,college_conference,draft_number
1715,2019.0,6.0,188.0,TEN,David Long,LB,22.0,2022.0,West Virginia,/players/L/LongDa04.htm,...,NaN,NaN,NaN,David Long,LB,5-11,224,2019.0,Big Twelve Conference,188.0
2190,2021.0,5.0,154.0,NYJ,Michael Carter,S,22.0,2022.0,Duke,/players/C/CartMi02.htm,...,NaN,NaN,NaN,Michael Carter,DB,5-10,190,2021.0,Atlantic Coast Conference,154.0


There were two David Longs and Michael Carters taken in their respective drafts so everything is correct

### Update target column to have players who were drafted in round 1 and players who were not

In [9]:
df.Rnd[df.Rnd > 1] = 0

C:\Users\capta\AppData\Local\Temp\ipykernel_17708\2056627649.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Rnd[df.Rnd > 1] = 0


In [10]:
df.Rnd.value_counts(normalize=True)

0.0    0.875293
1.0    0.124707
Name: Rnd, dtype: float64

### Unify the different versions of positions into their proper classes

In [11]:
df.Pos[df.Pos == 'NT'] = 'DT'
df.Pos[(df.Pos == 'T')|(df.Pos == 'OT')|(df.Pos == 'C')|(df.Pos == 'G')] = 'OL'
df.Pos[df.Pos == 'SAF'] = 'S'
df.Pos[(df.Pos == 'OLB')|(df.Pos == 'ILB')] = 'LB'

C:\Users\capta\AppData\Local\Temp\ipykernel_17708\1793448353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Pos[df.Pos == 'NT'] = 'DT'
C:\Users\capta\AppData\Local\Temp\ipykernel_17708\1793448353.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Pos[(df.Pos == 'T')|(df.Pos == 'OT')|(df.Pos == 'C')|(df.Pos == 'G')] = 'OL'
C:\Users\capta\AppData\Local\Temp\ipykernel_17708\1793448353.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [12]:
df.Pos.value_counts()

OL    426
LB    320
WR    319
DE    223
RB    215
CB    210
DT    191
DB    173
TE    144
S     138
QB    113
DL     27
P      20
K      17
FB     14
LS      8
Name: Pos, dtype: int64

### Check other possible position misclassification 

In [13]:
df[df.Pos == 'DB']

,Draft_Yr,Rnd,Pick,Tm,Player,Pos,Age,To,College/Univ,Player_NFL_Link,...,pass_int,pass_rating,g,display_name,position_group,height,weight,entry_year,college_conference,draft_number
8,2013.0,1.0,9.0,NYJ,Dee Milliner,DB,21.0,2015.0,Alabama,/players/M/MillDe00.htm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,2013.0,1.0,12.0,OAK,D.J. Hayden,DB,23.0,2021.0,Houston,/players/H/HaydDJ00.htm,...,NaN,NaN,NaN,D.J. Hayden,DB,5-11,190,2013.0,Conference USA,12.0
14,2013.0,1.0,15.0,NOR,Kenny Vaccaro,DB,22.0,2020.0,Texas,/players/V/VaccKe00.htm,...,NaN,NaN,NaN,Kenny Vaccaro,DB,6-0,214,2013.0,Big Twelve Conference,15.0
17,2013.0,1.0,18.0,SFO,Eric Reid,DB,21.0,2019.0,LSU,/players/R/ReidEr00.htm,...,NaN,NaN,NaN,Eric Reid,DB,73,215,2013.0,Southeastern Conference,18.0
21,2013.0,1.0,22.0,ATL,Desmond Trufant,DB,22.0,2021.0,Washington,/players/T/TrufDe00.htm,...,NaN,NaN,NaN,Desmond Trufant,DB,6-0,190,2013.0,Pacific Twelve Conference,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2531,2022.0,0.0,236.0,LAC,Deane Leonard,DB,22.0,2022.0,Mississippi,/players/L/LeonDe00.htm,...,NaN,NaN,NaN,Deane Leonard,DB,6-2,195,2022.0,Southeastern Conference,236.0
2532,2022.0,0.0,237.0,DET,Chase Lucas,DB,25.0,2022.0,Arizona St.,/players/L/LucaCh01.htm,...,NaN,NaN,NaN,Chase Lucas,DB,6-0,176,2022.0,Pacific Twelve Conference,237.0
2538,2022.0,0.0,243.0,KAN,Jaylen Watson,DB,23.0,2022.0,Washington St.,/players/W/WatsJa02.htm,...,NaN,NaN,NaN,Jaylen Watson,DB,6-3,204,2022.0,Pacific Twelve Conference,243.0
2539,2022.0,0.0,244.0,ARI,Christian Matthew,DB,25.0,2022.0,Valdosta St.,/players/M/MattCh01.htm,...,NaN,NaN,NaN,Christian Matthew,DB,6-4,200,2022.0,Gulf South Conference,244.0


DB and CB differ enough as DB's are more of a hybrid slot CB and S combo while CBs are pure CBs and mainly on the outside

### Finishing up some minor cleaning in renaming columns and changing the type of NFL Player Python Package 

In [14]:
# Remove / from column header
df = df.rename({'College/Univ':'College'},axis=1)

In [15]:
# Convert datatype of weight to numeric
df['weight'] = df.weight.astype('float64')

### Updated College Conference Data

In [16]:
df.college_conference.value_counts()

Southeastern Conference                             521
Big Ten Conference                                  348
Atlantic Coast Conference                           318
Pacific Twelve Conference                           289
Big Twelve Conference                               193
American Athletic Conference                        123
Mountain West Conference                             84
Conference USA                                       75
Mid-American Conference                              61
Independent                                          60
Sun Belt Conference                                  38
Missouri Valley Football Conference                  31
Colonial Athletic Association                        18
Big East                                             14
Big Sky Conference                                   13
Ohio Valley Conference                               13
Southern Conference                                  12
Mid-America Intercollegiate Athletic Association

### Correct any data that misclassifies the conference due to conference realignment following the 2014 season

In [17]:
df.college_conference[(df.college_conference.isna())&(df.College == 'Maryland')&(df.Draft_Yr<2014)] = 'Atlantic Coast Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_17708\1625936797.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.college_conference[(df.college_conference.isna())&(df.College == 'Maryland')&(df.Draft_Yr<2014)] = 'Atlantic Coast Conference'


In [18]:
df.college_conference[(df.college_conference.isna())&(df.College == 'Maryland')&(df.Draft_Yr>=2014)] = 'Big Ten Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_17708\3846577269.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.college_conference[(df.college_conference.isna())&(df.College == 'Maryland')&(df.Draft_Yr>=2014)] = 'Big Ten Conference'


In [19]:
df.college_conference[(df.college_conference.isna())&(df.College == 'Rutgers')&(df.Draft_Yr>=2014)] = 'Big Ten Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_17708\2555060057.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.college_conference[(df.college_conference.isna())&(df.College == 'Rutgers')&(df.Draft_Yr>=2014)] = 'Big Ten Conference'


In [20]:
df.college_conference[(df.college_conference.isna())&(df.College == 'Rutgers')&(df.Draft_Yr<2014)] = 'Atlantic Coast Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_17708\3990046507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.college_conference[(df.college_conference.isna())&(df.College == 'Rutgers')&(df.Draft_Yr<2014)] = 'Atlantic Coast Conference'


In [21]:
df.college_conference[(df.College == 'Rutgers')&(df.Draft_Yr<2014)] = 'Atlantic Coast Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_17708\2265236477.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.college_conference[(df.College == 'Rutgers')&(df.Draft_Yr<2014)] = 'Atlantic Coast Conference'


### Manually add college conferences for rows that are missing it

In [22]:
sec = ['Alabama','Florida','LSU','South Carolina','Arkansas','Vanderbilt','Texas A&M','Auburn',]
acc = ['Boston College', 'Clemson', 'Florida St.', 'Louisville', 'North Carolina St.', 'Syracuse', 'Wake Forest', 
       'Duke', 'Georgia Tech','Miami (FL)', 'North Carolina', 'Pittsburgh', 'Virginia', 'Virginia Tech']
aac = ['Cincinnati','Connecticut','East Carolina','South Florida','Temple','Central Florida',
       'Houston','Memphis','Navy','Southern Methodist','Tulane','Tulsa']
big12 = ['Baylor','Iowa St.','Kansas','Kansas St.','Oklahoma','Oklahoma St.','TCU',
         'Texas','Texas Tech','West Virginia']
big10 = ['Indiana','Maryland','Michigan','Michigan St.','Ohio St.','Penn St.',
         'Rutgers','Illinois','Iowa','Minnesota','Nebraska','Northwestern','Purdue','Wisconsin']
cusa = ['Charlotte','East Carolina','Florida Atlantic','Florida International','Marshall','Middle Tennessee',
        'Old Dominion','Western Kentucky','Louisiana Tech','North Texas','Rice','Southern Miss','Ala-Birmingham','UTEP','UTSA']

In [23]:
missing_conf = df[(df.college_conference.isna())]
missing_conf.College.unique()

array(['Alabama', 'Florida St.', 'Florida', 'Wisconsin', 'North Carolina',
       'West Virginia', 'LSU', 'South Carolina', 'Valdosta St.',
       'Arkansas', 'Fresno St.', 'USC', 'Kansas St.', 'Connecticut',
       'UCLA', 'Louisiana Tech', 'Georgia', 'Michigan', 'Stanford',
       'Clemson', 'California', 'Western Kentucky', 'Miami (FL)',
       'Illinois', 'Richmond', 'Vanderbilt', 'Colorado', 'Texas A&M',
       'Notre Dame', 'Oklahoma', 'Oregon', 'Jacksonville St.', 'Nevada',
       'New Mexico St.', 'Grand Valley St.', 'Southern Utah', 'Samford',
       'Ohio', 'Chadron St.', 'Harding', 'Elon', 'New Hampshire',
       'Florida A&M', 'Iowa St.', 'Duke', 'Massachusetts', 'Oklahoma St.',
       'Boise St.', 'Oregon St.', 'Auburn', 'Kent St.', 'Minnesota',
       'San Jose St.', 'Syracuse', 'Maine', 'Wyoming', 'Baylor',
       'Marshall', 'Arizona St.', 'Georgia Tech', 'Indiana',
       'Portland St.', 'Boston Col.', 'Texas Tech', 'Ohio St.',
       'Missouri', 'Georgia Southern', 'S

In [24]:
df.columns

Index(['Draft_Yr', 'Rnd', 'Pick', 'Tm', 'Player', 'Pos', 'Age', 'To',
       'College', 'Player_NFL_Link', 'Player_NCAA_Link', 'tackles_solo',
       'tackles_assists', 'tackles_total', 'tackles_loss', 'sacks', 'def_int',
       'def_int_yds', 'def_int_yds_per_int', 'def_int_td', 'pass_defended',
       'fumbles_rec', 'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced',
       'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'rush_att', 'rush_yds',
       'rush_yds_per_att', 'rush_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'pass_cmp', 'pass_att', 'pass_cmp_pct',
       'pass_yds', 'pass_yds_per_att', 'adj_pass_yds_per_att', 'pass_td',
       'pass_int', 'pass_rating', 'g', 'display_name', 'position_group',
       'height', 'weight', 'entry_year', 'college_conference', 'draft_number'],
      dtype='object')

### Save the cleaned data for EDA

In [25]:
df.to_parquet('./Data/draft1322')

### Drop additional columns to allow for the modeling to be done

In [26]:
df.drop(['entry_year','draft_number','g','Player_NFL_Link','Player_NCAA_Link','display_name','position_group'],axis=1,inplace=True)

In [27]:
df.columns

Index(['Draft_Yr', 'Rnd', 'Pick', 'Tm', 'Player', 'Pos', 'Age', 'To',
       'College', 'tackles_solo', 'tackles_assists', 'tackles_total',
       'tackles_loss', 'sacks', 'def_int', 'def_int_yds',
       'def_int_yds_per_int', 'def_int_td', 'pass_defended', 'fumbles_rec',
       'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced', 'rec', 'rec_yds',
       'rec_yds_per_rec', 'rec_td', 'rush_att', 'rush_yds', 'rush_yds_per_att',
       'rush_td', 'scrim_att', 'scrim_yds', 'scrim_yds_per_att', 'scrim_td',
       'pass_cmp', 'pass_att', 'pass_cmp_pct', 'pass_yds', 'pass_yds_per_att',
       'adj_pass_yds_per_att', 'pass_td', 'pass_int', 'pass_rating', 'height',
       'weight', 'college_conference'],
      dtype='object')

### Save new modeling file

In [28]:
df.to_parquet('./Data/modeldraft1322')